<a href="https://colab.research.google.com/github/DM872/Material/blob/main/FeaturesSel/feat_sel.ipynb">,
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"\>,
    </a>

# Task 2


## Solution Approach



### Sets and Indices

$i \in I=\{1,2,\dots,n\}$: Set of observations.

$j \in J=\{0,1,2,\dots,p\}$: Set of features, where the first ID corresponds to the intercept.

$\ell \in L = J \setminus \{0\}$: Set of features, where the intercept is excluded.

### Parameters

$s \in \mathbb{N}$: Number of features to include in the model, ignoring the intercept.


### Decision Variables

$\beta_j \in \mathbb{R}$: Weight of feature $j$, representing the change in the response variable per unit-change of feature $j$.

$z_\ell \in \{0,1\}$: 1 if weight of feature $\ell$ is exactly equal to zero, and 0 otherwise. Auxiliary variable used to manage the budget constraint.

### Objective Function

- **Training error**: Minimize the Sum of Abosulte Residuals (aka L1) more robust with respect to outliers:

\begin{equation}
\text{Min} \quad Z = \sum_{i \in I}\left|y_i-\sum_{j \in J}\beta_{j}x_{ij}\right|
\end{equation}


### Constraints

- **Mutual exclusivity**: For each feature $\ell$, if $z_\ell=1$, then $\left|\beta_\ell\right|=0$

\begin{equation}
\left|\beta_\ell\right|\leq M (1-z_\ell)
\end{equation}

Here the big-M is the upper bound of the $\beta$ variables, which are actually unlimited. We could introduce an artificial upper bound and we could add the lasso term in the objective function:

$$
\begin{equation}
\text{Min} \quad Z = \sum_{i \in I}\left|y_i-\sum_{j \in J}\beta_{j}x_{ij}\right| + \lambda \sum_{\ell=1}^p\left|\beta\right|
\end{equation}
$$

Alternatively we can define as a Special Ordered Set of type one (meaning that at most one variables is different from zero (variables can be both integer of continuous):

\begin{equation}
(\beta_\ell, z_\ell): \text{SOS-1} \quad \forall \ell \in L
\tag{1}
\end{equation}

- **Budget constraint**: Exactly $|L| - s$ feature weights must be equal to zero:

\begin{equation}
\sum_{\ell \in L}z_\ell = |L| - s
\tag{2}
\end{equation}

This model, by means of constraint 2, implicitly considers all ${{p} \choose s}$ feature subsets at once. However, we also need to find the value for $s$ that maximizes the performance of the regression on unseen observations. Notice that the training error decreases monotonically as more features are considered, so it is not advisable to use it as the performance metric. Instead, we should estimate the Mean Squared Error (MSE) via cross-validation. This metric is defined as $\text{MSE}=\frac{1}{n}\sum_{i=1}^{n}{(y_i-\hat{y}_i)^2}$, where $y_i$ and $\hat{y}_i$ are the observed and predicted values for the ith observation, respectively. Then, we will fine-tune $s$ using grid search, provided that the set of possible values is quite small.


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math

import gurobipy as gp
from gurobipy import GRB

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [2]:
# Load data and split into train (80%) and test (20%)
boston = load_boston()
X = boston['data']
y = boston['target']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2,random_state=10101)

In [3]:
scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain_std = scaler.transform(Xtrain)
Xtest_std = scaler.transform(Xtest)

In [4]:
Xtrain_std.shape[1]

13

In [5]:
m = gp.Model("fitting")

# Sets
p = Xtrain_std.shape[1] # n of features
n = Xtrain_std.shape[0] # n of observations (data points)

I=range(n)
J=range(p+1)
L=range(1,p+1)

# Parameters
M=10000
s=5 # features in the model

# Create the decision variables
beta = {}
for j in J:
    beta[j] = m.addVar(lb=-float('inf'), vtype=GRB.CONTINUOUS, name=f"beta_{j}")

zeta = {}
for j in L:
    zeta[j] = m.addVar(vtype=GRB.BINARY, name="zeta"+str(j))
    
# auxiliary to handle absolut values
betap = m.addVars(L, lb=0.0, vtype=GRB.CONTINUOUS, name="betap")
betan = m.addVars(L, lb=0.0, vtype=GRB.CONTINUOUS, name="betan")
        
errp = m.addVars(I, lb=0.0, vtype=GRB.CONTINUOUS, name="errn")
errn = m.addVars(I, lb=0.0, vtype=GRB.CONTINUOUS, name="errp")


# The objective is to minimize 
m.modelSense=gp.GRB.MINIMIZE
m.setObjective(gp.quicksum(errp[i]+errn[i] for i in range(n)))


for i in I:
    m.addConstr(errp[i]-errn[i]==ytrain[i]-beta[0]-gp.quicksum(beta[j]*Xtrain_std[i][j-1] for j in L ))

    
m.addConstr(gp.quicksum(zeta[j] for j in L)>=p-s)

for j in L:
    m.addConstr(betap[j]-betan[j]==beta[j])
    m.addConstr(betap[j]+betan[j]<=M*(1-zeta[j]))

#for j in L:
    # If zeta[i]=1, then beta[i] = 0
    # m.addSOS(GRB.SOS_TYPE1, [zeta[j], beta[j]])
    

# Solve
m.write("feat_sel.lp")
#m.display()
m.optimize()


if m.status == gp.GRB.status.OPTIMAL:
    print('\nSum of Absolute Residuals (Training Error): %g' % m.ObjVal)
    print('\nbetas:')
    for j in range(p+1):
        if math.fabs(beta[j].X)>0+0.0001:
            print('%s %g' % (j, beta[j].X))
    print('\nzetas:')
    for j in range(1,p+1):
        #if zeta[j].X>0+0.0001:
        print('%s %g' % (j, zeta[j].X))
else:
    print('No solution')


Academic license - for non-commercial use only - expires 2021-05-06
Using license file /Users/marco/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 431 rows, 861 columns and 6555 nonzeros
Model fingerprint: 0x0cd87ed7
Variable types: 848 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 1e+04]
Found heuristic solution: objective 9112.8000000
Presolve time: 0.02s
Presolved: 431 rows, 861 columns, 6555 nonzeros
Variable types: 848 continuous, 13 integer (13 binary)

Root relaxation: objective 1.248587e+03, 451 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1248.58702    0    9 9112.80000 1248.58702